In [2]:
import numpy as np
import os
import pandas as pd
from typing import Dict

## Descrição do problema

Determine a temperatura uniforme de equilíbrio $T_{placa}$ de uma placa vertical (altura $3.0m$; comprimento $6.0m$). Uma de suas faces (e) é exposta ao ar ambiente $T_{\infty}=310K$ e recebe um fluxo prescrito $q_e”=800\frac{W}{m^2}$, enquanto a outra face (d) recebe um fluxo prescrito $q_d”=140\frac{W}{m^2}$.

In [ ]:
plate_dimensions : Dict = {'height' : 3.0, 'lenght' : 6.0}
prescribed_heat : Dict = {'side_e' : 800, 'side_d' : 140}
enviroment_temperature : np.float128 = 310.0

## Propriedades do Ar 
Para alimentar o código iterativo que irá calcular o coeficiente convectivo do problema foram extraídos os valores do livro para as propriedades termofisicas e utilizando esses valores foram construídos interpoladores que tem como varíavel de controle o valor da temperatura de filme $T_F$

In [27]:
class air:
    def __init__(self, air_conf : str = os.path.join(os.getcwd(),'air_params_treated.parquet')) -> None:
        self.data = pd.read_parquet(air_conf)
        (self.data)
    def air_parameters(self, T_f : np.float128 ) -> Dict:
        out =  { key : np.interp( x = T_f, xp = self.data['T'].to_list(), fp = self.data[key].to_list()) for key in self.data.loc[:, self.data.columns != 'T'].columns}
        out['Tf'] = T_f
        return out
print(air().air_parameters(300))

{'Ro': 1.1614, 'Cp': 1.007, 'mu': 1.846e-05, 'v': 1.589e-05, 'k': 0.0263, 'alpha': 2.2499999999999998e-05, 'Pr': 0.707, 'Tf': 300}


In [29]:
class Hc():
    def __init__(self) -> None:
        self.air_config : air = air()
        self.g : np.float128 = 9.8
    def __Beta__(self, air_params : Dict)->np.float128:
        return 1/air_params['Tf']
    def __Grashof__(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310)-> np.float128:
        return (self.g*self.__Beta__(air_params)*(T_s - T_inf)*np.float_power(L,3))/np.float_power(air_params['v'],2)
    def __Rayleigh__(self,air_params : np.float128, L: np.float128, T_s = 410, T_inf = 310):
        return self.__Grashof__(air_params, L, T_s, T_inf)*air_params['Pr']

True


## Modelagem do Problema
### Balanço de Energia
Como descrito no problema, a temperatura de equílibrio uniforme foi solicitada, isso significa que a energia armazenada no sistema deve ser igual a zero. Somando isso ao fato que não foi explicitado no problema que a placa gera algum tipo de calor temos que o balanço de enrgia pode ser descrito da seguinte forma:
$$ E_{entra} - E_{sai} = 0$$
Sendo que:
$$ E_{entra} = q_{e}"\cdot A + q_{d}"\cdot A = \left(q_{e}" + q_{d}"\right)\cdot A $$
$$ E_{sai} = h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) $$
Logo temos  que :
$$ \left(q_{e}" + q_{d}"\right)\cdot A - h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) = 0 $$
Como o valor solicitado é a a temperatura da placa tem-se que :
$$ T_{placa} = \frac{q_{e}" + q_{d}"}{h_{c}} + T_{\infty} $$
Fazendo a substituição dos valores conhecidos temos que :
$$ T_{placa} = \frac{800 + 140}{h_{c}} + 310 $$
Temos então que o resultado pode ser descrito como sendo :
$$ T_{placa} = \frac{940}{h_{c}} + 310 $$
### Condição de Parada
A partir do que foi descrito acima podemos afirmar que o código deve parar quando a seguinte equação for satisfeita:
$$\left(q_{e}" + q_{d}"\right)\cdot A - h_{c}\cdot A\left(T_{placa} - T_{\infty}\right) = 0 $$
$$\left(q_{e}" + q_{d}"\right) - h_{c}\left(T_{placa} - T_{\infty}\right) = 0 $$
### Espaço de Busca
Para ser possível definir os valores aceitáveis de $T_{placa}$ é necessário avaliar a equação. Tendo em mente que $h_c$ sempre será um número positivo entre 0 e 1 e que todos os valores conhecidos são números reais e positivos já é possível afirmar algumas condições:
1. $T_{placa}$  deve ser maior que zero.
2. $T_{placa}$ deve ser maior de $T_{\infty}$.
3. $\left(q_{e}" + q_{d}"\right)$ e $h_{c}\left(T_{placa} - T_{\infty}\right)$ devem ser iguais
4. O resultado do modelo independe da área $A$ da placa, porém depende da altura da placa.

Isso quer dizer que o problema possui soluções para o intervalo: 
$$ T_{\infty} < T_{placa} < \infty $$

In [ ]:
class Plate:
    def __init__(self,height : np.float128, lenght : np.float128) -> None:
        self.__dimensions__ = {'height':height, 'lenght':lenght}
    
    @property
    def area(self) -> np.float128:
        return self.__dimensions__['height']*self.__dimensions__['lenght']
    
    @property
    def height(self) -> np.float128:
        return self.__dimensions__['height']
    
    @property
    def lenght(self) -> np.float128:
        return self.__dimensions__['lenght']


## Modelo do Problema
Com o contexto definido do problema é possível escrever o problema da seguinte forma :
$$ \min f(T_{Placa}) = \left(q_{e}" + q_{d}"\right) - h_{c}\left(T_{placa} - T_{\infty}\right) $$
Para :
$$ T_{Placa} > T_{\infty} $$
$$ T_{Placa} \neq T_{\infty} $$

In [ ]:
def f_objective(Qe : np.float128, Qd : np.float128, Hc: np.float128, T_inf : str, T_placa):
    return (Qe+Qd)-(Hc*(T_placa - T_inf))
